In [1]:
import pandas as pd
import numpy as np
import os
print("✅ Libraries ready!")

✅ Libraries ready!


In [2]:
# Load the file
df = pd.read_csv(r'C:\Users\bheemanathi\Downloads\synthetic_ecommerce_data.csv')


print(f"✅ Loaded! Rows: {len(df):,}")
print(df.columns.tolist())

✅ Loaded! Rows: 100,000
['Transaction_ID', 'Customer_ID', 'Product_ID', 'Transaction_Date', 'Units_Sold', 'Discount_Applied', 'Revenue', 'Clicks', 'Impressions', 'Conversion_Rate', 'Category', 'Region', 'Ad_CTR', 'Ad_CPC', 'Ad_Spend']


In [3]:
rows_before = len(df)
df = df.drop_duplicates(subset=['Transaction_ID'])
df = df.dropna(subset=['Transaction_ID','Customer_ID','Transaction_Date','Revenue'])
df[['Discount_Applied','Clicks','Impressions','Ad_Spend','Ad_CTR','Ad_CPC']] = \
    df[['Discount_Applied','Clicks','Impressions','Ad_Spend','Ad_CTR','Ad_CPC']].fillna(0)
print(f"🗑️ Removed: {rows_before - len(df):,} rows | Remaining: {len(df):,}")

🗑️ Removed: 0 rows | Remaining: 100,000


In [4]:
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], dayfirst=True, errors='coerce')
df['Year']       = df['Transaction_Date'].dt.year
df['Month']      = df['Transaction_Date'].dt.month
df['Month_Name'] = df['Transaction_Date'].dt.strftime('%B')
df['Quarter']    = df['Transaction_Date'].dt.quarter
df['YearMonth']  = df['Transaction_Date'].dt.to_period('M').astype(str)
print(f"✅ Dates fixed! Range: {df['Transaction_Date'].min().date()} → {df['Transaction_Date'].max().date()}")

✅ Dates fixed! Range: 2023-07-12 → 2024-12-11


In [5]:
df = df[df['Revenue'] >= 0]
df = df[df['Units_Sold'] > 0]
df['Ad_Spend'] = df['Ad_Spend'].abs()
print(f"✅ Negatives removed! Rows: {len(df):,}")

✅ Negatives removed! Rows: 100,000


In [6]:
df['Category'] = df['Category'].str.strip().str.title()
df['Region']   = df['Region'].str.strip().str.title()
print("📦 Categories:", df['Category'].unique().tolist())
print("🌍 Regions:", df['Region'].unique().tolist())

📦 Categories: ['Electronics', 'Home Appliances', 'Toys', 'Clothing', 'Books']
🌍 Regions: ['Europe', 'Asia', 'North America']


In [7]:
df['Profit']            = df['Revenue'] - df['Ad_Spend']
df['Profit_Margin_Pct'] = ((df['Profit'] / df['Revenue']) * 100).round(2)
df['Revenue_Per_Unit']  = (df['Revenue'] / df['Units_Sold']).round(2)
df['ROAS']              = np.where(df['Ad_Spend'] > 0, (df['Revenue'] / df['Ad_Spend']).round(2), 0)
print("✅ New columns: Profit, Profit_Margin_Pct, Revenue_Per_Unit, ROAS")

✅ New columns: Profit, Profit_Margin_Pct, Revenue_Per_Unit, ROAS


In [8]:
print('='*50)
print('✅ FINAL DATASET SUMMARY')
print('='*50)
print(f'Total Rows       : {len(df):,}')
print(f'Unique Customers : {df["Customer_ID"].nunique():,}')
print(f'Unique Products  : {df["Product_ID"].nunique():,}')
print(f'Total Revenue    : ${df["Revenue"].sum():,.2f}')
print(f'Total Profit     : ${df["Profit"].sum():,.2f}')
print(f'Overall ROAS     : {(df["Revenue"].sum()/df["Ad_Spend"].sum()).round(2)}x')
print('='*50)

✅ FINAL DATASET SUMMARY
Total Rows       : 100,000
Unique Customers : 19,865
Unique Products  : 1,000
Total Revenue    : $55,311,081.07
Total Profit     : $44,263,357.10
Overall ROAS     : 5.01x


In [11]:
# STEP 1 — Check what the raw dates look like
print("Sample raw dates:")
print(df['Transaction_Date'].head(20))
print("\nNaT count:", df['Transaction_Date'].isna().sum())

Sample raw dates:
0    2024-06-10
1           NaT
2    2024-04-04
3           NaT
4    2024-05-05
5           NaT
6    2023-08-12
7           NaT
8    2024-10-04
9           NaT
10          NaT
11   2024-07-10
12   2024-06-07
13   2024-11-11
14          NaT
15          NaT
16          NaT
17          NaT
18   2024-01-08
19          NaT
Name: Transaction_Date, dtype: datetime64[ns]

NaT count: 60786


In [14]:
# Complete clean using df_raw (all 100,000 rows with dates fixed)

df = df_raw.copy()

# Remove duplicates
df = df.drop_duplicates(subset=['Transaction_ID'])

# Fill nulls
df[['Discount_Applied','Clicks','Impressions','Ad_Spend','Ad_CTR','Ad_CPC']] = \
    df[['Discount_Applied','Clicks','Impressions','Ad_Spend','Ad_CTR','Ad_CPC']].fillna(0)
df['Category'] = df['Category'].fillna('Unknown')
df['Region']   = df['Region'].fillna('Unknown')

# Remove negatives
df = df[df['Revenue'] >= 0]
df = df[df['Units_Sold'] > 0]
df['Ad_Spend'] = df['Ad_Spend'].abs()

# Fix dates — extract time columns
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce')
df['Year']       = df['Transaction_Date'].dt.year
df['Month']      = df['Transaction_Date'].dt.month
df['Month_Name'] = df['Transaction_Date'].dt.strftime('%B')
df['Quarter']    = df['Transaction_Date'].dt.quarter
df['YearMonth']  = df['Transaction_Date'].dt.to_period('M').astype(str)

# Standardize text
df['Category'] = df['Category'].str.strip().str.title()
df['Region']   = df['Region'].str.strip().str.title()

# New calculated columns
df['Profit']            = df['Revenue'] - df['Ad_Spend']
df['Profit_Margin_Pct'] = ((df['Profit'] / df['Revenue']) * 100).round(2)
df['Revenue_Per_Unit']  = (df['Revenue'] / df['Units_Sold']).round(2)
df['ROAS']              = np.where(df['Ad_Spend'] > 0, (df['Revenue'] / df['Ad_Spend']).round(2), 0)

# Final summary
print('='*50)
print('✅ FINAL CLEAN DATASET')
print('='*50)
print(f'Total Rows       : {len(df):,}')
print(f'Unique Customers : {df["Customer_ID"].nunique():,}')
print(f'Unique Products  : {df["Product_ID"].nunique():,}')
print(f'NaT in dates     : {df["Transaction_Date"].isna().sum()}')
print(f'Total Revenue    : ${df["Revenue"].sum():,.2f}')
print(f'Total Profit     : ${df["Profit"].sum():,.2f}')
print(f'Overall ROAS     : {(df["Revenue"].sum()/df["Ad_Spend"].sum()).round(2)}x')
print('='*50)

✅ FINAL CLEAN DATASET
Total Rows       : 100,000
Unique Customers : 19,865
Unique Products  : 1,000
NaT in dates     : 0
Total Revenue    : $55,311,081.07
Total Profit     : $44,263,357.10
Overall ROAS     : 5.01x


In [15]:
# Save final cleaned file
df.to_csv(r'C:\Users\bheemanathi\Downloads\ecommerce_cleaned.csv', index=False)
print("✅ Saved! ecommerce_cleaned.csv is ready!")

✅ Saved! ecommerce_cleaned.csv is ready!
